In [ ]:
%pip install toloka-kit==0.1.12
%pip install crowd-kit

In [14]:
import toloka.client as toloka
import toloka.client.project.template_builder as tb
from crowdkit.aggregation import MajorityVote, DawidSkene

In [4]:
import os
import json
import time
import random
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import pandas as pd
import numpy as np
import datetime

In [5]:
direct_xlsx = pd.read_excel('./direct_etpgpb.xlsx')

In [6]:
direct_xlsx

,Заголовок,Текст,Ссылка
0,Закупки и тендеры Москвы,На площадке ЭТП ГПБ тендеры Москвы представлен...,https://etpgpb.ru/
1,Закупки и тендеры,На площадке ЭТП ГПБ тендеры представлены от та...,https://etpgpb.ru/
2,Закупки и тендеры Москвы,На площадке ЭТП ГПБ тендеры Москвы представлен...,https://etpgpb.ru/landing/
3,Закупки и тендеры,На площадке ЭТП ГПБ тендеры представлены от та...,https://etpgpb.ru/landing/moskva/
4,Закупки и тендеры Москвы,На площадке ЭТП ГПБ тендеры Москвы представлен...,https://etpgpb.ru/regions/moskva/
5,Закупки и тендеры Москвы,На площадке ЭТП ГПБ тендеры Москвы представлен...,https://etpgpb.ru/portal/procedures/moskva/
6,Закупки и тендеры,На площадке ЭТП ГПБ тендеры представлены от та...,https://etpgpb.ru/landing/sankt-peterburg/
7,Закупки и тендеры Москвы,На площадке ЭТП ГПБ тендеры Москвы представлен...,https://etpgpb.ru/regions/sankt-peterburg/
8,Закупки и тендеры,На площадке ЭТП ГПБ тендеры представлены от та...,https://etpgpb.ru/portal/procedures/sankt-pete...
9,Закупки и тендеры Москвы,На площадке ЭТП ГПБ тендеры Москвы представлен...,https://etpgpb.ru/landing/leningradskaya-oblast/


### Простучим URL

In [7]:
def check_urls(url, max_retries = 3):
    session = requests.Session()
    retry = Retry(total=max_retries, backoff_factor=0.1, status_forcelist=[500, 502, 503, 504])
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.9',
        'Accept-Encoding': 'gzip, deflate, br',
        'Referer': 'https://etpgpb.ru/',
        'DNT': '1',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1'
    }
    
    time.sleep(3)

    try:
        response = session.get(url, headers=headers, timeout=45)
        response.raise_for_status()
        return str(response.status_code)
    except requests.exceptions.HTTPError as e:
        return str(response.status_code)
    except requests.exceptions.ConnectionError as e:
        print(f"An error occurred: {e}")
        return None
    except requests.exceptions.Timeout as e:
        print(f"An error occurred: {e}")
        return None

    
direct_xlsx["Ответ сервера"] = direct_xlsx["Ссылка"].apply(check_urls)

In [8]:
#результаты простукивания
direct_xlsx

,Заголовок,Текст,Ссылка,Ответ сервера
0,Закупки и тендеры Москвы,На площадке ЭТП ГПБ тендеры Москвы представлен...,https://etpgpb.ru/,200
1,Закупки и тендеры,На площадке ЭТП ГПБ тендеры представлены от та...,https://etpgpb.ru/,200
2,Закупки и тендеры Москвы,На площадке ЭТП ГПБ тендеры Москвы представлен...,https://etpgpb.ru/landing/,200
3,Закупки и тендеры,На площадке ЭТП ГПБ тендеры представлены от та...,https://etpgpb.ru/landing/moskva/,404
4,Закупки и тендеры Москвы,На площадке ЭТП ГПБ тендеры Москвы представлен...,https://etpgpb.ru/regions/moskva/,200
5,Закупки и тендеры Москвы,На площадке ЭТП ГПБ тендеры Москвы представлен...,https://etpgpb.ru/portal/procedures/moskva/,200
6,Закупки и тендеры,На площадке ЭТП ГПБ тендеры представлены от та...,https://etpgpb.ru/landing/sankt-peterburg/,404
7,Закупки и тендеры Москвы,На площадке ЭТП ГПБ тендеры Москвы представлен...,https://etpgpb.ru/regions/sankt-peterburg/,200
8,Закупки и тендеры,На площадке ЭТП ГПБ тендеры представлены от та...,https://etpgpb.ru/portal/procedures/sankt-pete...,200
9,Закупки и тендеры Москвы,На площадке ЭТП ГПБ тендеры Москвы представлен...,https://etpgpb.ru/landing/leningradskaya-oblast/,404


### Посмотрим что скажет GPT
**подготовим все для работы YaGPT**

In [9]:
#получаем iam_token
def get_llm_token(token):
    r = requests.post('https://iam.api.cloud.yandex.net/iam/v1/tokens', 
              headers = {'Content-Type': 'application/x-www-form-urlencoded'}, 
              data = '{\"yandexPassportOauthToken\":\"'+token+'\"}').text
    return json.loads(r)["iamToken"]

iam_token = get_llm_token(os.environ['model_oauth_token'])

#делаем запрос к модели
def prompt_yagpt(prompt):

    h = {"Authorization": "Bearer " + str(iam_token), 
         "x-folder-id": os.environ['model_dir_id']}
    data= {
    "modelUri": 'gpt://'+os.environ['model_dir_id']+'/yandexgpt/latest',
    "completionOptions": {
    "temperature": 0.1,
    "maxTokens": "10000"
    },
    "messages": [
    {
      "role": "user",
      "text": prompt
    }]}
    yagpt_request = requests.post('https://llm.api.cloud.yandex.net/foundationModels/v1/completion', headers = h, json = data)
    #print(yagpt_request.json())
    return yagpt_request.json()['result']['alternatives'][0]['message']['text']

prompt_yagpt("Привет! Это тест!)")

'Привет! Это не тест, а начало диалога. Что хотите обсудить?'

**сформируем промпт**

In [10]:
def is_text_url_relevant(title, description, url):
    time.sleep(5)
    prompt = '''Ты работаешь с контекстной рекламой. 
    Посмотри на json с информацией о рекламном объявлении и ответь "релевантны ли title и description к url?"
    Ответь только в формате json, такого вида'''
    prompt += json.dumps({"answer":1, "comment":"Релевантно, потому что..."}, ensure_ascii = False)
    prompt += '''где answer бинарный ответ 1 - релевантны, 0 - нерелевантны, comment - краткое понятное объяснение почему
    
    далее json который нужно посмотреть:'''
    prompt += json.dumps({"title":title, "description":description, "url":url}, ensure_ascii = False)
    return json.loads(prompt_yagpt(prompt))

is_text_url_relevant("Закупки и тендеры Москвы", "На площадке ЭТП ГПБ тендеры Москвы представлены от таких крупных игроков рынка как ГК Газпром, Уралмашзавод, АО «МРТС»", "https://etpgpb.ru/portal/procedures/leningradskaya-oblast/")

{'answer': 0,
 'comment': 'Не подходит, так как объявление не связано с тендерами в Ленинградской области.'}

In [11]:
def apply_relevance(row):
    result = is_text_url_relevant(row["Заголовок"], row["Текст"], row["Ссылка"])
    return pd.Series([result['answer'], result['comment']], index=["Релевантность Ответ", "Релевантность Комментарий"])

# Применяем функцию ко всем строкам DataFrame и добавляем новые столбцы
direct_xlsx[["Релевантность Ответ", "Релевантность Комментарий"]] = direct_xlsx.apply(apply_relevance, axis=1)


In [12]:
#результаты проверки релевантности от YaGPT
direct_xlsx

,Заголовок,Текст,Ссылка,Ответ сервера,Релевантность Ответ,Релевантность Комментарий
0,Закупки и тендеры Москвы,На площадке ЭТП ГПБ тендеры Москвы представлен...,https://etpgpb.ru/,200,1,"Релевантно, потому что в тексте объявления ука..."
1,Закупки и тендеры,На площадке ЭТП ГПБ тендеры представлены от та...,https://etpgpb.ru/,200,1,"Релевантно, потому что в тексте объявления ука..."
2,Закупки и тендеры Москвы,На площадке ЭТП ГПБ тендеры Москвы представлен...,https://etpgpb.ru/landing/,200,1,"Релевантно, потому что объявление соответствуе..."
3,Закупки и тендеры,На площадке ЭТП ГПБ тендеры представлены от та...,https://etpgpb.ru/landing/moskva/,404,1,"Релевантно, потому что объявление соответствуе..."
4,Закупки и тендеры Москвы,На площадке ЭТП ГПБ тендеры Москвы представлен...,https://etpgpb.ru/regions/moskva/,200,1,"Релевантно, потому что в тексте объявления ука..."
5,Закупки и тендеры Москвы,На площадке ЭТП ГПБ тендеры Москвы представлен...,https://etpgpb.ru/portal/procedures/moskva/,200,1,"Релевантно, потому что в тексте объявления ука..."
6,Закупки и тендеры,На площадке ЭТП ГПБ тендеры представлены от та...,https://etpgpb.ru/landing/sankt-peterburg/,404,1,"Релевантно, потому что объявление соответствуе..."
7,Закупки и тендеры Москвы,На площадке ЭТП ГПБ тендеры Москвы представлен...,https://etpgpb.ru/regions/sankt-peterburg/,200,0,"Не подходит, так как в тексте объявления речь ..."
8,Закупки и тендеры,На площадке ЭТП ГПБ тендеры представлены от та...,https://etpgpb.ru/portal/procedures/sankt-pete...,200,1,"Релевантно, потому что в тексте объявления ука..."
9,Закупки и тендеры Москвы,На площадке ЭТП ГПБ тендеры Москвы представлен...,https://etpgpb.ru/landing/leningradskaya-oblast/,404,0,"Не подходит, так как объявление не связано с Л..."


### Посмотрим что скажут исполнители Яндекс.Заданий
**подключимся к заданиям**

In [22]:
toloka_client = toloka.TolokaClient(os.environ['tasks_token'], url = 'https://tasks.yandex.ru')
print(toloka_client.get_requester())

Requester(_unexpected={}, id='c5e281f2771b0ed61c7c53dede710068', balance=Decimal('2908.5619'), public_name={'EN': 'ШАД-99'}, company=Requester.Company(_unexpected={}, id='151', superintendent_id='64ef54a8b2a44461537f06b121533c42'))


In [16]:
new_pool = toloka_client.clone_pool("3203665")
new_pool.id

'3204602'

In [17]:
tasks = [
    toloka.Task(
        input_values={
            'title': row['Заголовок'],
            'description': row['Текст'],
            'url': row['Ссылка']
        },
        pool_id=new_pool.id
    )
    for index, row in direct_xlsx.iterrows()
]

In [18]:
upload_tasks = toloka_client.create_tasks(tasks, allow_defaults=True)
print("Заданий загружено " + str(len(upload_tasks.items)))

Заданий загружено 16


In [19]:
new_pool = toloka_client.open_pool(new_pool.id)
pool_id = new_pool.id

def wait_pool_for_close(pool_id, minutes_to_wait=1):
    sleep_time = 60 * minutes_to_wait
    pool = toloka_client.get_pool(pool_id)
    while not pool.is_closed():
        op = toloka_client.get_analytics([toloka.analytics_request.CompletionPercentagePoolAnalytics(subject_id=pool.id)])
        op = toloka_client.wait_operation(op)
        percentage = op.details['value'][0]['result']['value']
        print(
            f'   {datetime.datetime.now().strftime("%H:%M:%S")}\t'
            f'Пул {pool.id} - {percentage}%'
        )
        time.sleep(sleep_time)
        pool = toloka_client.get_pool(pool.id)
    print('Pool was closed.')

wait_pool_for_close(pool_id)

   17:58:17	Пул 3204602 - 0%
   17:59:19	Пул 3204602 - 56%
   18:00:20	Пул 3204602 - 88%
   18:01:22	Пул 3204602 - 94%
   18:02:23	Пул 3204602 - 94%
   18:03:24	Пул 3204602 - 94%
   18:04:25	Пул 3204602 - 94%


KeyboardInterrupt: 

In [20]:
answers_df = toloka_client.get_assignments_df(pool_id)

Experimental method


In [21]:
answers_df.sample(10)

,INPUT:url,INPUT:title,INPUT:description,OUTPUT:result,GOLDEN:result,HINT:text,HINT:default_language,ASSIGNMENT:link,ASSIGNMENT:assignment_id,ASSIGNMENT:worker_id,ASSIGNMENT:status,ASSIGNMENT:started
41,https://etpgpb.ru/portal/procedures/moskva/,Закупки и тендеры Москвы,На площадке ЭТП ГПБ тендеры Москвы представлен...,ok,NaN,NaN,NaN,https://tasks.yandex.ru/task/3204602/000030e5f...,000030e5fa--66df375ce1bdd4210f0ce579,38b0e2fad2196d54cb96f57cee302643,APPROVED,2024-09-09T17:58:52.494
28,https://etpgpb.ru/industry/sooruzheniya-i-stro...,Закупки и тендеры Москвы,На площадке ЭТП ГПБ тендеры Москвы представлен...,bad,NaN,NaN,NaN,https://tasks.yandex.ru/task/3204602/000030e5f...,000030e5fa--66df374d71ef98386efd3ff4,5e26fd58a8193b0a86f88404cf57ba5e,APPROVED,2024-09-09T17:58:37.218
47,https://etpgpb.ru/regions/moskovskaya-oblast/,Закупки и тендеры Московской области,На площадке ЭТП ГПБ тендеры Московской области...,ok,NaN,NaN,NaN,https://tasks.yandex.ru/task/3204602/000030e5f...,000030e5fa--66df3762ca13f361a4d64992,9f13a5b4fc9252d54094b1579ee3bf43,APPROVED,2024-09-09T17:58:58.685
50,https://etpgpb.ru/regions/moskva/,Закупки и тендеры Москвы,На площадке ЭТП ГПБ тендеры Москвы представлен...,ok,NaN,NaN,NaN,https://tasks.yandex.ru/task/3204602/000030e5f...,000030e5fa--66df37b2ca13f361a4d650ba,c3ba229b1d3d5963cf98a5a80ecdca48,APPROVED,2024-09-09T18:00:18.558
9,https://etpgpb.ru/industry/sooruzheniya-i-stro...,Закупки и тендеры Москвы,На площадке ЭТП ГПБ тендеры Москвы представлен...,ok,NaN,NaN,NaN,https://tasks.yandex.ru/task/3204602/000030e5f...,000030e5fa--66df3747336a0a23c400085a,f4be96835765b440a5902769be4a425e,APPROVED,2024-09-09T17:58:31.219
45,https://etpgpb.ru/landing/leningradskaya-oblast/,Закупки и тендеры Москвы,На площадке ЭТП ГПБ тендеры Москвы представлен...,bad,NaN,NaN,NaN,https://tasks.yandex.ru/task/3204602/000030e5f...,000030e5fa--66df3762ca13f361a4d64992,9f13a5b4fc9252d54094b1579ee3bf43,APPROVED,2024-09-09T17:58:58.685
24,https://etpgpb.ru/,Закупки и тендеры Москвы,На площадке ЭТП ГПБ тендеры Москвы представлен...,ok,NaN,NaN,NaN,https://tasks.yandex.ru/task/3204602/000030e5f...,000030e5fa--66df374b38e58d70b5016775,4e2adfb57e6c6663fd635a16102b0535,APPROVED,2024-09-09T17:58:35.628
46,https://etpgpb.ru/,Закупки и тендеры,На площадке ЭТП ГПБ тендеры представлены от та...,ok,NaN,NaN,NaN,https://tasks.yandex.ru/task/3204602/000030e5f...,000030e5fa--66df3762ca13f361a4d64992,9f13a5b4fc9252d54094b1579ee3bf43,APPROVED,2024-09-09T17:58:58.685
29,https://etpgpb.ru/,Закупки и тендеры Москвы,На площадке ЭТП ГПБ тендеры Москвы представлен...,bad,NaN,NaN,NaN,https://tasks.yandex.ru/task/3204602/000030e5f...,000030e5fa--66df374d71ef98386efd3ff4,5e26fd58a8193b0a86f88404cf57ba5e,APPROVED,2024-09-09T17:58:37.218
2,https://etpgpb.ru/landing/moskovskaya-oblast/,Закупки и тендеры Москвы,На площадке ЭТП ГПБ тендеры Москвы представлен...,bad,NaN,NaN,NaN,https://tasks.yandex.ru/task/3204602/000030e5f...,000030e5fa--66df37436afae908824e6c56,0b6220c79eb6b1fb984a992637c464d3,APPROVED,2024-09-09T17:58:27.239


In [ ]:
answer_mv_df = pd.DataFrame()
answer_mv_df['task'] = answers_df["INPUT:url"] + ";" + answers_df["INPUT:title"] + ";" +answers_df["INPUT:description"]

In [ ]:
answer_mv_df